In [2]:
import os
import time

import duckdb
import httpx
import pandas as pd
import matplotlib.pyplot as plt
from retry import retry
from dotenv import load_dotenv


load_dotenv()

True

In [4]:
import json


def write_json(data, filename='data.json'):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [3]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def get_transaction(network, collection_id, continuation="", type=["sale", "transfer", "mint"], last_timestamp: int = -1):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url = f"https://{networks[network]}.reservoir.tools/collections/activity/v6"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    params = {
        "collection": collection_id,
        "limit": 50,
        "types": type,
    }

    if continuation != "":
        params["continuation"] = continuation

    resp = httpx.get(url, params=params, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    resp = resp.json()
    
    if last_timestamp != -1:
        resp["activities"] = [activity for activity in resp["activities"] if activity["timestamp"] > last_timestamp]
    
    if len(resp["activities"]) == 0:
        resp["continuation"] = None
    
    for activity in resp["activities"]:
        activity["network"] = network
        activity["collection_id"] = collection_id
        
    return resp

In [ ]:
resp = get_transaction("ethereum", "0x06012c8cf97bead5deae237070f9587f8e7a266d", type=["mint"])
write_json(resp, "output/mint3.json")

In [5]:
resp = get_transaction("ethereum", "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d", type=["mint"], last_timestamp=1668322751)
resp

{'activities': [], 'continuation': None}

In [64]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def get_transfer_bulk(network, collection_id, continuation="", start_timestamp=-1, end_timestamp=-1):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url = f"https://{networks[network]}.reservoir.tools/transfers/bulk/v2"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    params = {
        "contract": collection_id,
        "limit": 1000,
    }

    if continuation != "":
        params["continuation"] = continuation
        
    if start_timestamp != -1:
        params["startTimestamp"] = start_timestamp
    
    if end_timestamp != -1:
        params["endTimestamp"] = end_timestamp

    resp = httpx.get(url, params=params, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    resp = resp.json()
    for transfer in resp["transfers"]:
        transfer["network"] = network
        transfer["collection_id"] = collection_id
    

    return resp

In [ ]:
resp = get_transfer_bulk("ethereum", "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d", start_timestamp=1718693423)
write_json(resp, "output/transfer_bulk.json")

In [67]:
start_timestamp = 1718693423
end_timestamp = int(time.time())
continuation = ""

while True:
    resp = get_transfer_bulk("ethereum", "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d", start_timestamp=start_timestamp, end_timestamp=end_timestamp, continuation=continuation)
    write_json(resp, f"output/transfer_bulk_{start_timestamp}_{continuation}.json")
    
    continuation = resp["continuation"]
    
    if continuation in [None, ""]:
        break

In [ ]:
df = duckdb.read_json("output/*.json")

In [ ]:
transfers = df["transfers"].fetchdf()
transfers

In [ ]:
flatten = pd.concat([pd.json_normalize(row["transfers"]) for idx, row in transfers.iterrows()])

In [ ]:
flatten

In [ ]:
flatten.shape

In [ ]:
# Assuming the timestamps are stored in the 'timestamp' column of the 'flatten' DataFrame
timestamps = flatten['timestamp']

# Plotting the histogram of timestamps
plt.hist(timestamps)
plt.xlabel('Timestamp')
plt.ylabel('Frequency')
plt.title('Distribution of Timestamps')
plt.show()

In [ ]:
flatten.duplicated().sum()

In [ ]:
1718693423

In [ ]:
time.time()

In [ ]:
time.time() - 1718693423

In [ ]:
(time.time() - 1718693423) / 60 / 60 / 24

In [ ]:
flatten.loc[flatten["timestamp"] < 1718769000]

In [ ]:
flatten.iloc[0].txHash

In [ ]:
flatten["token.tokenId"].value_counts()

In [ ]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def get_sale(network, collection_id, continuation="", start_timestamp=-1, end_timestamp=-1):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url = f"https://{networks[network]}.reservoir.tools/sales/v6"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    params = {
        "collection": collection_id,
        "limit": 1000
    }

    if continuation != "":
        params["continuation"] = continuation

    if start_timestamp != -1:
        params["startTimestamp"] = start_timestamp
    
    if end_timestamp != -1:
        params["endTimestamp"] = end_timestamp
    
    resp = httpx.get(url, params=params, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    resp = resp.json()

    for sale in resp["sales"]:
        sale["network"] = network
        sale["collection_id"] = collection_id
    
    return resp

In [ ]:
resp = get_sale("ethereum", "0x06012c8cf97bead5deae237070f9587f8e7a266d")
write_json(resp, "output/sale.json")

In [ ]:
resp = get_transaction("ethereum", "0x06012c8cf97bead5deae237070f9587f8e7a266d", type=["sale"])
write_json(resp, "output/sale2.json")